In [ ]:
#Data preprocessing
!python3 MT-Preparation/filtering/filter.py TED2020.en-tr.tr TED2020.en-tr.en tr en

In [ ]:
!python3 MT-Preparation/train_dev_split/train_dev_test_split.py 10000 10000 TED2020.en-tr.tr-filtered.tr TED2020.en-tr.en-filtered.en

In [3]:
fp = open("Data-train-src.txt", "w")
with open("TED2020.en-tr.tr-filtered.tr.train", 'r') as file:
    for i in range(300000):
        fp.write(file.readline())

fp.close()

fp = open("Data-train-src.txt", "a")
with open("kk_train_shuffled.txt-filtered.kk", 'r') as file:
    for i in range(300000):
        fp.write(file.readline())

fp.close()

In [4]:
fp = open("Data-train-tgt.txt", "w")
with open("TED2020.en-tr.en-filtered.en.train", 'r') as file:
    for i in range(300000):
        fp.write(file.readline())

fp.close()

fp = open("Data-train-tgt.txt", "a")
with open("en_train_shuffled.txt-filtered.en", 'r') as file:
    for i in range(300000):
        fp.write(file.readline())

fp.close()

In [ ]:
#Kazakh and Turkish shared vocabulary
!onmt-build-vocab --sentencepiece model_type=bpe --size 32000 --save_vocab kk_tr_shared_vocab Data-train-src.txt

In [ ]:
#English side shared vocabulary
!onmt-build-vocab --sentencepiece model_type=bpe --size 32000 --save_vocab en_shared_vocab Data-train-tgt.txt

In [7]:
# Kk-En -> Tr-En (TED2020)
!onmt-main --model kk-tr-en-shared.py --config data.yml --auto_config train --num_gpus 2

2024-12-11 10:10:07.047375: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-11 10:10:07.912337: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-12-11 10:10:07.912403: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2024-12-11 10:10:07.912411: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like t

In [8]:
# Kk-En -> Tr-En (TED2020)
!onmt-main --model kk-tr-en-shared.py --config kk-tr-en-2.yml --auto_config --checkpoint_path KK-TR-EN-Shared-vocab/ckpt-100000 train --with_eval --num_gpus 2

2024-12-13 10:49:32.500231: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-13 10:49:33.331780: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-12-13 10:49:33.331846: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2024-12-13 10:49:33.331854: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like t

In [9]:
!CUDA_VISIBLE_DEVICES=0 onmt-main --config kk-tr-en-2.yml --auto_config --checkpoint_path KK-TR-EN-Shared-vocab-2/ckpt-100000 infer --features_file TED2020_tokens_test_shared --predictions_file output_kk_en_tr_shared_vocab.txt

2024-12-14 13:41:33.726880: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-14 13:41:34.892107: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-12-14 13:41:34.892308: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2024-12-14 13:41:34.892322: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like t

In [ ]:
!python3 MT-Preparation/subwording/3-desubword.py en_shared_vocab.model output_kk_en_tr_shared_vocab.txt

In [1]:
# BLEU and chrF scores
!python3 compute-bleu.py TED2020.en-tr.en-filtered.en.test output_kk_en_tr_shared_vocab.txt.desubword

Reference first sentence: It's quite widespread.
Translated first sentence: So this is a very common topic .
That's 100 lines that end in a tokenized period ('.')
It looks like you forgot to detokenize your test data, which may hurt your score.
If you insist your data is detokenized, or don't care, you can suppress this message with the `force` parameter.
BLEU2:  BLEU = 24.21 55.6/30.1/18.2/11.3 (BP = 1.000 ratio = 1.070 hyp_len = 211818 ref_len = 197923)
CHRF:  chrF2 = 51.36


In [12]:
# Average METEOR score (Ortalama METEOR Puanı)
import nltk
from nltk.translate.meteor_score import meteor_score

def read_and_tokenize_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    return [nltk.word_tokenize(line.strip()) for line in lines]

def calculate_meteor(reference_file, hypothesis_file):
    references = read_and_tokenize_file(reference_file)
    hypotheses = read_and_tokenize_file(hypothesis_file)
    
    if len(references) != len(hypotheses):
        raise ValueError("Dosyaların satır sayıları eşleşmiyor")

    total_meteor_score = 0.0
    for ref, hyp in zip(references, hypotheses):
        total_meteor_score += meteor_score([ref], hyp)

    average_meteor_score = total_meteor_score / len(references)
    return average_meteor_score

reference_file = 'TED2020.en-tr.en-filtered.en.test'
hypothesis_file = 'output_kk_en_tr_shared_vocab.txt.desubword'

score = calculate_meteor(reference_file, hypothesis_file)
print(f"Ortalama METEOR Puanı: {score}")

Ortalama METEOR Puanı: 0.5690762834709019


In [13]:
# Tr-En (TED2020) -> Kk-En
!onmt-main --model kk-tr-en-shared.py --config tr-en-shared-vocab.yml --auto_config train --with_eval --num_gpus 2

2024-12-14 13:48:20.986162: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-14 13:48:21.796544: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-12-14 13:48:21.796612: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2024-12-14 13:48:21.796621: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like t

In [14]:
# Tr-En (TED2020) -> Kk-En
!onmt-main --model kk-tr-en-shared.py --config Tr-Kk-En_shared_vocab.yml --auto_config --checkpoint_path TR-EN-Shared-vocab/ckpt-100000 train --with_eval --num_gpus 2

2024-12-15 16:15:15.056020: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-15 16:15:15.850608: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-12-15 16:15:15.850678: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2024-12-15 16:15:15.850686: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like t

In [15]:
!CUDA_VISIBLE_DEVICES=0 onmt-main --config Tr-Kk-En_shared_vocab.yml --auto_config --checkpoint_path TR-KK-EN-Shared-vocab/ckpt-100000 infer --features_file KK_tokens_test_shared --predictions_file output_tr_kk_en_shared_vocab.txt

2024-12-16 10:52:53.847923: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-16 10:52:54.646980: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-12-16 10:52:54.647050: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2024-12-16 10:52:54.647058: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like t

In [ ]:
!python3 MT-Preparation/subwording/3-desubword.py en_shared_vocab.model output_tr_kk_en_shared_vocab.txt

In [2]:
# BLEU and chrF scores
!python3 compute-bleu.py en_test_shuffled.txt-filtered.en output_tr_kk_en_shared_vocab.txt.desubword

Reference first sentence: In the developed world, this figure is 35 25%
Translated first sentence: In developed countries of the world , this figure is 35% 25%
That's 100 lines that end in a tokenized period ('.')
It looks like you forgot to detokenize your test data, which may hurt your score.
If you insist your data is detokenized, or don't care, you can suppress this message with the `force` parameter.
BLEU2:  BLEU = 52.06 72.4/56.3/46.3/38.9 (BP = 1.000 ratio = 1.118 hyp_len = 463010 ref_len = 414303)
CHRF:  chrF2 = 76.62


In [18]:
# Average METEOR score (Ortalama METEOR Puanı)
import nltk
from nltk.translate.meteor_score import meteor_score

def read_and_tokenize_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    return [nltk.word_tokenize(line.strip()) for line in lines]

def calculate_meteor(reference_file, hypothesis_file):
    references = read_and_tokenize_file(reference_file)
    hypotheses = read_and_tokenize_file(hypothesis_file)
    
    if len(references) != len(hypotheses):
        raise ValueError("Dosyaların satır sayıları eşleşmiyor")

    total_meteor_score = 0.0
    for ref, hyp in zip(references, hypotheses):
        total_meteor_score += meteor_score([ref], hyp)

    average_meteor_score = total_meteor_score / len(references)
    return average_meteor_score

reference_file = 'en_test_shuffled.txt-filtered.en'
hypothesis_file = 'output_tr_kk_en_shared_vocab.txt.desubword'

score = calculate_meteor(reference_file, hypothesis_file)
print(f"Ortalama METEOR Puanı: {score}")

Ortalama METEOR Puanı: 0.7636741748442291
